# Part 2 - Softmax Regression

The goal of this part of the project is to implement Softmax Regression in order to classify the MNIST digit dataset. Softmax Regression is essentially a two-layer neural network where the output layer applies the Softmax cost function, a multiclass generalization of the logistic cost function.

In logistic regression, we have a hypothesis function of the form

$$\mathbb{P}[y=1] = \frac{1}{1 + e^{-wx}}$$

where $\mathbf{w}$ is our weight vector. Like the hyperbolic tangent function, the logistic function is also a sigmoid function with the characteristic 's'-like shape, though it has a range of $(0, 1)$ instead of $(-1, 1)$. Note that this is technically not a classifier since it returns probabilities instead of a predicted class, but it's easy to turn it into a classifier by simply choosing the class with the highest probability.

Since logistic regression is used for binary classification, it is easy to see that:

$$\begin{aligned}
\mathbb{P}[y=1] &= \frac{1}{1 + e^{-wx}} \\
&= \frac{e^{wx}}{e^{wx} + e^{0 \cdot x}} \\
\end{aligned}$$

Similarily,

$$\begin{aligned}
\mathbb{P}[y=0] &= 1 - \mathbb{P}[y=1] \\
&= 1 - \frac{e^{wx}}{1 + e^{wx}} \\
&= \frac{e^{0 \cdot x}}{e^{wx} + e^{0 \cdot x}} \\
\end{aligned}$$

From this form it appears that we can assign the vector $w_1 = w$ as the weight vector for class $1$ and $w_0 = \vec{0}$ as the weight vector for class $0$. This motivates generalization to classification with more than 2 classes. By assigning a separate weight vector $w_i$ to each class, for each example $x$ we can predict the probability that it is class $i$, and again we can classify by choosing the most probable class.

$$\mathbb{P}[y=i] = \frac{e^{w_i x}}{\sum\limits_j e^{w_j x}}$$

A more compact way of representing the values $w_i x$ is $Wx$ where each row $i$ of $W$ is $w_i$. We can also represent a dataset $\{x_i\}$ with a matrix $X$ where each column is a single example, resulting in $WX$.

## Part 2.1 - Questions [5%]

For both problems, assume there are $C$ classes, $n$ be the number of samples, and $d$ be the number of features for each sample. 

1. Prove that the probabilities outputed by the softmax function sum to 1. 

**ANSWER**:

2. Given the description of matrices $W$, $X$ above, what are the dimensions of $W$, $X$, and $WX$? 

**ANSWER**:

## Part 2.2 - Implementing a Softmax Classifier [15%]
We can also train on this model with an appropriate loss function. The Softmax loss function is given by

$$\xi(W) = -\sum\limits_{i=1}^{n}\sum\limits_{j=1}^{C}\delta_{y_i,j}\log \mathbb{P}(y_i = j)$$

where $n$ is the number of samples, $C$ is the number of classes, $\delta_{y_i,j}$ is the [kronecker delta](https://en.wikipedia.org/wiki/Kronecker_delta), $y_i$ is the label of sample $i$, and $\mathbb{P}(y_i = j)$ is calculated using a softmax function using the weights from matrix $W$.

Note that 

$$\delta_{y_i,j} = \begin{cases}1 &\text{if } y_i = j \\ 0 & \text{if } y_i \neq j\end{cases}$$

The gradient of this function is given by 

$$\nabla_{\vec{w_j}} \xi(W) = \sum_{i=1}^{n}{ \left[ x_i \left( \mathbb{P}[y_i = j] - \delta_{y_i, j} \right) \right]}$$

The derivation for this can be found in the references. Note that the kronecker delta and the probabilities can be represented as matrices, which makes the code for the loss and the gradient very simple.

In [ ]:
import numpy as np
import scipy.sparse
import scipy.optimize

class SoftmaxRegression:
    """
    Here you will fill in this incomplete implementation of Softmax regression.

    Adapted from code by Jatin Shah
    """
    def __init__(self, num_classes, ex_size, opts={'maxIter':400}):
        """
        num_classes:    number of possible classifications
        ex_size:        size of attribute array (number of input features)
        reg:            regularizing term coefficient (lambda)
        opts:           in this class the only option used is maxIter
        """

        self.num_classes = num_classes
        self.ex_size = ex_size
        self.opts = opts

        # Initialize weight matrix with empty matrix
        self.W = np.zeros((num_classes, ex_size))


    def cost(self, X, Y, W=None):
        """
        Calculate the cost function for X and Y using current weight matrix W. Note that we are not using
        a regularizer in the cost; this is equivalent to lambda = 0.

        X:              (M x N) matrix of input feature values,
                            where M = ex_size, N = number of examples
        Y:              (N x 1) array of expected output classes for each example

        Returns the cost and its gradient, which is the form needed to use scipy.optimize.minimize
        """

        if W is None:
            W = this.W
        num_classes = self.num_classes
        ex_size = self.ex_size

        W = W.reshape(num_classes, ex_size)         # Ensure W is in the correct dimensions
        N = X.shape[1]                              # N = number of examples

        W_X = W.dot(X)                              # This is our activation matrix with dimensions (A * B)
                                                    # where A is the number of classes and B is the number
                                                    # of examples. (W_X[a, b] gives the activation of example
                                                    # b for class a.) You will use this matrix to find the
                                                    # probabilities that example b is class a using the
                                                    # softmax formula.

        W_X = W_X - np.amax(W_X)

        # This is the indicator function used in the loss function, where indicator[a, b] = 1
        # when example b is labeled a (according to the target Y) and indicator[a, b] = 0 otherwise.

        indicator = scipy.sparse.csr_matrix((np.ones(N), (Y, np.array(range(N)))))
        indicator = np.resize(np.array(indicator.todense()), (num_classes, N))


        # Each column of W_X is the set of activations for each class corresponding to
        # one example; the probabilties are given by the exponential of each entry
        # divided by the sum of the exponentials over the entire column.

        # The cost associated with a single example is given by -1 times the log probability
        # of the true class; initialize the cost variable to the AVERAGE cost over all the examples.
        # Hint: there's an easy way to do this with the indicator matrix.

        # The gradient has the same dimensions as W, and each component (i,j) represents the
        # derivative of the cost with respect to the weight associated with class i, attribute j.
        # The gradient associated with a single example x is given by -1 * A * x_T, where x_T is
        # the transpose of the example, and A is a vector with component i given by (1 - P(class = i))
        # if the true class is i, and (-P(class = i)) otherwise. Notice that this multiplication gives
        # the desired dimensions. Find the AVERAGE gradient over all the examples. Again, there is
        # an easy way to do this with the indicator matrix.

        # TODO: Compute the predicted probabilities, the total cost, and the gradient.

        ### YOUR CODE HERE ###

        raise NotImplementedError
        # probabilities =
        # cost =
        # gradient =

        ### YOUR CODE (ENDS) HERE ###

        # flatten is needed by scipy.optimize.minimize
        return cost, gradient.flatten()


    def train(self, X, Y):
        """
        Train to find optimal weight matrix W. Here we make use of the SciPy optimization library but
        in theory you could implement gradient descent to do this as well.

        X:              (M x N) matrix of input feature values,
                            where M = ex_size, N = number of examples
        Y:              (N x 1) array of expected output classes for each example
        maxIter:        Maximum training iterations
        """

        num_classes = self.num_classes
        ex_size = self.ex_size
        W = self.W

        # Set maxIter hyperparameter
        if self.opts['maxIter'] is None:
            self.opts['maxIter'] = 400

        # Lambda function needed by scipy.optimize.minimize
        J = lambda w: self.cost(X, Y, w)

        result = scipy.optimize.minimize(J, W, method='L-BFGS-B', jac=True, options={'maxiter': self.opts['maxIter'], 'disp': True})
        self.W = result.x # save the optimal solution found

    def predict(self, X):
        """
        Use W to predict the classes of each example in X.

        X:              (M x N) matrix of input feature values,
                            where M = ex_size, N = number of examples

        """

        W = self.W.reshape(self.num_classes, self.ex_size)
        W_X = W.dot(X)

        # TODO: Compute the predicted probabilities and the predicted classes for each example
        # Reminder: The predicted class for a single example is just the one with the highest probability

        ### YOUR CODE HERE ###

        raise NotImplementedError
        # probabilities =
        # predicted_classes =

        ### YOUR CODE (ENDS) HERE ###

        return predicted_classes

You can test your implementation of `SoftmaxRegression` by running the following snippet below:

In [ ]:
def good_job(path):
    a = plt.imread(urllib.request.urlopen(path))
    fig = plt.imshow(a)
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.show()

def loadMNIST(image_file, label_file):
    """
    returns a 28x28x[number of MNIST images] matrix containing
    the raw MNIST images
    :param filename: input data file
    """

    with open(image_file, "r") as f:
        magic = np.fromfile(f, dtype=np.dtype('>i4'), count=1)

        num_images = np.fromfile(f, dtype=np.dtype('>i4'), count=1)[0]
        num_rows = np.fromfile(f, dtype=np.dtype('>i4'), count=1)[0]
        num_cols = np.fromfile(f, dtype=np.dtype('>i4'), count=1)[0]

        exSize = num_rows * num_cols
        images = np.fromfile(f, dtype=np.ubyte)
        images = images.reshape((num_images, exSize)).transpose()
        images = images.astype(np.float64) / 255

        f.close()

    with open(label_file, 'r') as f:
        magic = np.fromfile(f, dtype=np.dtype('>i4'), count=1)

        num_labels = np.fromfile(f, dtype=np.dtype('>i4'), count=1)

        labels = np.fromfile(f, dtype=np.ubyte)

        f.close()

    return images, labels

ex_size = 28*28   # size of MNIST digits
num_classes = 10  # 10 digits to classify

X, Y = loadMNIST('data/train-images.idx3-ubyte', 'data/train-labels.idx1-ubyte')
sm = SoftmaxRegression(num_classes, ex_size)
sm.train(X, Y)

testX, testY = loadMNIST('data/t10k-images.idx3-ubyte', 'data/t10k-labels.idx1-ubyte')
predictions = sm.predict(testX)
accuracy = 100 * np.sum(predictions == testY, dtype=np.float64) / testY.shape[0]
print(f"Accuracy: {accuracy:.2f}")
assert accuracy >= 93.99, "Your implementation is sadly incorrect :("

print("TEST PASSED!!!")
good_job("https://www.cs.umd.edu/~tomg/img/important_memes/hacker_database.png")

## Part 2.3 - Stability

In the `cost` function of `SoftmaxRegression`, we see the line

```python3
W_X = W_X - np.amax(W_X)
```

1. What is this operation doing?

**ANSWER**:

2. Show that this does not affect the predicted probabilities.

**ANSWER**:

3. Why might this be an optimization? Justify your answer.

**ANSWER**:

## 2.4 - Analysis of Classifier Accuracy [10%]

Plot the accuracy of the classifier as a function of the number of examples seen.
Do you observe any overfitting or underfitting? Discuss and expain what you observe. Use the helper functions provided below.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def train_test(classifier, X, Y, Xtest, Ytest):
    """
    Train a classifier on data (X,Y) and evaluate on
    data (Xtest,Ytest).  Return a tuple of:
      * Training data accuracy
      * Test data accuracy
    """
    classifier.train(X, Y)
    train_acc = np.mean(Y == classifier.predict(X))
    test_acc = np.mean(Ytest == classifier.predict(Xtest))

    print(f"Training accuracy {train_acc}, test accuracy {test_acc}")

    return train_acc, test_acc

def learning_curve(classifier, X, Y, Xtest, Ytest):
    """
    Generate a learning curve by repeatedly halving the amount of
    training data until none is left.

    We return a triple containing:
      * The sizes of data sets we trained on
      * The training accuracies at each level
      * The test accuracies at each level
    """

    N = X.shape[1]                   # how many total points?
    M = int(np.ceil(np.log2(N)))     # how many classifiers will we have to train?

    data_sizes = np.zeros(M)
    train_accs  = np.zeros(M)
    test_accs   = np.zeros(M)

    for i in range(1, M+1):
        # select every 2^(M-i)th point
        ids = np.arange(0, N, 2**(M-i))
        Xtr = X[:, ids]
        Ytr = Y[ids]

        # report what we're doing
        print(f"Training classifier on {ids.size} points...")

        # train the classifier
        train_acc, test_acc = train_test(classifier, Xtr, Ytr, Xtest, Ytest)

        # store the results
        data_sizes[i-1] = ids.size
        train_accs[i-1] = train_acc
        test_accs[i-1]  = test_acc

    return (data_sizes, train_accs, test_accs)